#### Full citation
UNDP, Human Development Report (2024) – with minor processing by Our World in Data. “Human Development Index – UNDP” [dataset]. UNDP, Human Development Report, “Human Development Report 2023-2024” [original data].
Source: UNDP, Human Development Report (2024) – with minor processing by Our World In Data

In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = "path/to/human-development-index.csv"
df = pd.read_csv(csv_file_path)

# Group data by year
years = df["Year"].unique()

# Define the output Excel file path
excel_file_path = "path/to/HDI_process.xlsx"

# Create an ExcelWriter object and write different sheets
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    for year in sorted(years):
        df_year = df[df["Year"] == year].sort_values(by="Entity")
        df_year.to_excel(writer, sheet_name=str(year), index=False)

print(f"Excel file has been successfully generated: {excel_file_path}")

In [ ]:
import pandas as pd

# Load the processed Excel file
excel_file_path = "path/to/HDI_process.xlsx"

# Read the Excel file
xls = pd.ExcelFile(excel_file_path)

# Select the 2022 data sheet
df_2022 = pd.read_excel(xls, sheet_name='2022')

# Print the first 10 rows of the 2022 data
print("2022 Data (First 10 Rows):")
print(df_2022.head(10))

# Select Switzerland's data and sort by year
df_switzerland = pd.concat([pd.read_excel(xls, sheet_name=sheet) for sheet in xls.sheet_names])
df_switzerland = df_switzerland[df_switzerland["Entity"] == "Switzerland"].sort_values(by="Year")

# Print the first 10 rows of Switzerland's sorted data
print("Switzerland Data (Sorted by Year, First 10 Rows):")
print(df_switzerland.head(10))


2022 Data (First 10 Rows):
                Entity Code  Year  Human Development Index
0          Afghanistan  AFG  2022                 0.462000
1               Africa  NaN  2022                 0.556653
2              Albania  ALB  2022                 0.789000
3              Algeria  DZA  2022                 0.745000
4              Andorra  AND  2022                 0.884000
5               Angola  AGO  2022                 0.591000
6  Antigua and Barbuda  ATG  2022                 0.826000
7   Arab States (UNDP)  NaN  2022                 0.704000
8            Argentina  ARG  2022                 0.849000
9              Armenia  ARM  2022                 0.786000
Switzerland Data (Sorted by Year, First 10 Rows):
          Entity Code  Year  Human Development Index
132  Switzerland  CHE  1990                    0.850
132  Switzerland  CHE  1991                    0.852
132  Switzerland  CHE  1992                    0.855
132  Switzerland  CHE  1993                    0.859
132  Swit

In [17]:
import hvplot.pandas
import holoviews as hv
from holoviews import opts

# Plot scatter plot for 2022 data
scatter_plot = df_2022.hvplot.scatter(
    x='Entity', y='Human Development Index', c='Human Development Index', cmap='Category20', 
    rot=90, height=500, width=900, title='Human Development Index for 2022', colorbar=True
)

# Display the plot
scatter_plot

:Scatter   [Entity]   (Human Development Index)

In [31]:
import holoviews as hv

# Select four countries' data and sort by year
selected_countries = ["Switzerland", "China", "United States", "Egypt"]
df_selected = pd.concat([pd.read_excel(xls, sheet_name=sheet) for sheet in xls.sheet_names])
df_selected = df_selected[df_selected["Entity"].isin(selected_countries)].sort_values(by=["Year", "Entity"])

# Pivot the dataframe to get 'Year' as index and selected countries as columns
df_pivot = df_selected.pivot(index='Year', columns='Entity', values='Human Development Index')
df_pivot.reset_index(inplace=True)

# Print the first 10 rows of the pivoted dataframe
print("Four countries' data:")
print(df_pivot.head(10))

# Plot line chart for selected countries' Human Development Index over the years
line_plot = df_pivot.hvplot.line(
    x='Year', y=["Switzerland", "China", "United States", "Egypt"], 
    ylabel='Human Development Index', title='Human Development Index Over the Years', 
    height=500, width=900, legend='top'
)

# Display the plots
line_plot

Four countries' data:
Entity  Year  China  Egypt  Switzerland  United States
0       1990  0.482  0.567        0.850          0.875
1       1991  0.489  0.574        0.852          0.875
2       1992  0.501  0.580        0.855          0.880
3       1993  0.512  0.587        0.859          0.882
4       1994  0.522  0.594        0.861          0.885
5       1995  0.532  0.601        0.863          0.886
6       1996  0.543  0.610        0.867          0.887
7       1997  0.554  0.615        0.872          0.889
8       1998  0.565  0.620        0.878          0.891
9       1999  0.575  0.625        0.881          0.891


:NdOverlay   [Entity]
   :Curve   [Year]   (value)

In [36]:
import geoviews as gv
import geopandas as gpd
from holoviews import opts

df_2022['Human Development Index'] = pd.to_numeric(df_2022['Human Development Index'], errors='coerce')

# Load world map data from Natural Earth alternative source
world = gpd.read_file("https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_110m_admin_0_countries.geojson")

# Merge the 2022 HDI data with world map data
df_merged = world.merge(df_2022, left_on='ADMIN', right_on='Entity', how='left')

# Drop rows with missing HDI values
df_merged = df_merged.dropna(subset=['Human Development Index'])

# Plot world map with Human Development Index values and tooltips
world_map = gv.Polygons(df_merged, vdims=['Human Development Index', 'Entity']).opts(
    color='Human Development Index', cmap='viridis', line_color='black',
    width=900, height=500, title='Human Development Index (2022)', colorbar=True,
    tools=['hover'], hover_fill_color='white'
)

# Display the world map
world_map

:Polygons   [Longitude,Latitude]   (Human Development Index,Entity)